In [ ]:
%load_ext autoreload
%autoreload 2
%load_ext nb_black
%load_ext lab_black

<IPython.core.display.Javascript object>

In [ ]:
# default_exp model

<IPython.core.display.Javascript object>

# Model

In [ ]:
# hide
from nbdev.showdoc import *

<IPython.core.display.Javascript object>

In [ ]:
#export
import gc
import uuid
import joblib
import numpy as np
from tqdm.auto import tqdm
from pathlib import Path
from typeguard import typechecked
from abc import ABC, abstractmethod
from rich import print as rich_print
import lightgbm as lgb
from catboost import CatBoostRegressor
from sklearn.dummy import DummyRegressor

from numerai_blocks.dataset import Dataset
from numerai_blocks.preprocessing import display_processor_info

<IPython.core.display.Javascript object>

## 0. Base

In [ ]:
#export
@typechecked
class BaseModel(ABC):
    def __init__(self, model_directory: str, file_suffix: str, model_name: str = None, *args, **kwargs):
        self.model_directory = Path(model_directory)
        self.__dict__.update(*args, **kwargs)
        self.model_name = model_name if model_name else uuid.uuid4().hex
        self.prediction_col_name = f"prediction_{self.model_name}"
        self.description = f"{self.__class__.__name__}: '{self.model_name}' prediction"

        self.file_suffix = file_suffix
        self.model_paths = self.model_directory.glob(f'*.{self.file_suffix}')
        self.total_models = len(self.model_paths)
        if self.file_suffix:
            assert self.model_paths, f"No {self.file_suffix} files found in {self.model_directory}."

    @abstractmethod
    def predict(self, dataset: Dataset) -> Dataset:
        """ Return Dataset with column added for prediction. """
        ...
        return Dataset(**dataset.__dict__)

    def __call__(self, dataset: Dataset) -> Dataset:
        return self.predict(dataset)

<IPython.core.display.Javascript object>

## 1. Standard model formats

### 1.1. Joblib

In [ ]:
#export
@typechecked
class JoblibModel(BaseModel):
    """ Load in arbitrary models saved as .joblib. """
    def __init__(self, model_directory: str, model_name: str = None, *args, **kwargs):
        file_suffix = "joblib"
        super(JoblibModel, self).__init__(model_directory=model_directory,
                                          file_suffix=file_suffix,
                                          model_name=model_name,
                                          *args, **kwargs)

    @display_processor_info
    def predict(self, dataset: Dataset, *args, **kwargs) -> Dataset:
        dataset.dataf.loc[:, self.prediction_col_name] = np.zeros(len(dataset.dataf))
        feature_df = dataset.get_feature_data
        models = self._load_models()
        for model in tqdm(models, desc=self.description):
            predictions = model.predict(feature_df, *args, **kwargs)
            dataset.dataf.loc[:, self.prediction_col_name] += predictions
        del models; gc.collect()
        return Dataset(**dataset.__dict__)

    def _load_models(self) -> list:
        return [joblib.load(path) for path in self.model_paths]

<IPython.core.display.Javascript object>

### 1.2. Catboost (.cbm)

In [ ]:
#export
@typechecked
class CatboostModel(BaseModel):
    """ Predict using .cbm models (CatBoostRegressor) with all models in directory. """
    def __init__(self, model_directory: str, model_name: str = None, *args, **kwargs):
        file_suffix = "cbm"
        super(CatboostModel, self).__init__(model_directory=model_directory,
                                            file_suffix=file_suffix,
                                            model_name=model_name, *args, **kwargs)

    @display_processor_info
    def predict(self, dataset: Dataset) -> Dataset:
        dataset.dataf.loc[:, f"prediction_{self.model_name}"] = 0
        feature_df = dataset.get_feature_data
        models = self._load_models()
        for model in tqdm(models, desc=self.description):
            predictions = model.predict(feature_df)
            dataset.dataf.loc[:, self.model_name] += predictions
        del models; gc.collect()
        return Dataset(**dataset.__dict__)

    def _load_models(self) -> list:
        return [CatBoostRegressor().load_model(path) for path in self.model_paths]

<IPython.core.display.Javascript object>

### 1.3. LightGBM (.lgb)

In [ ]:
#export
@typechecked
class LGBMModel(BaseModel):
    """ Parse predict .cbm models (CatBoostRegressor) from directory. """
    def __init__(self, model_directory: str, model_name: str = None, *args, **kwargs):
        file_suffix = "lgb"
        super(LGBMModel, self).__init__(model_directory=model_directory,
                                        file_suffix=file_suffix,
                                        model_name=model_name, *args, **kwargs)

    @display_processor_info
    def predict(self, dataset: Dataset) -> Dataset:
        dataset.dataf.loc[:, f"prediction_{self.model_name}"] = 0
        feature_df = dataset.get_feature_data
        models = self._load_models()
        for model in tqdm(models, desc=self.description):
            predictions = model.predict(feature_df)
            dataset.dataf.loc[:, self.model_name] += predictions
        del models; gc.collect()
        return Dataset(**dataset.__dict__)

    def _load_models(self) -> list:
        return [lgb.Booster(path) for path in self.model_paths]

<IPython.core.display.Javascript object>

### 2. Baseline models

In [ ]:
class ConstantModel(BaseModel):
    def __init__(self, model_directory: str, constant_value: float = 0.5):
        self.constant_value = constant_value
        file_suffix = ""
        model_name = f"constant_{self.constant_value}"
        super(ConstantModel, self).__init__(model_directory=model_directory,
                                            file_suffix=file_suffix,
                                            model_name=model_name)
        self.clf = DummyRegressor(strategy='constant', constant=0.5)

    def predict(self, dataset: Dataset) -> Dataset:
        feature_df = dataset.get_feature_data
        dataset.dataf.loc[:, f"prediction_{self.model_name}"] = self.clf.predict(feature_df)
        return Dataset(**dataset.__dict__)

<IPython.core.display.Javascript object>

### 3. Custom Model

Arbitrary models can be instantiated by inheriting from `BaseModel` and defining the `file_suffix`. All model_paths of models that are in `model_directory` will be defined upon instantiation. Arbitrary logic (model loading, prediction, etc.) can be defined in `.predict` as long as the method takes a `Dataset` as input and outputs a `Dataset`. The Model should be able to typecheck by adding the `@typeguard.typechecked` decorator at the top of the class.

If loading model files is not relevant for your use case you should specify `file_suffix`="".

For clear console output we recommend adding the `@display_processor_info` decorator.

In [ ]:
#export
@typechecked
class AwesomeModel(BaseModel):
    """
    - TEMPLATE -
    Predict with arbitrary model formats.
    """
    def __init__(self, model_directory: str, model_name: str = None, *args, **kwargs):
        file_suffix = "anything"
        super(AwesomeModel, self).__init__(model_directory=model_directory,
                                           file_suffix=file_suffix,
                                           model_name=model_name, *args, **kwargs)

    @display_processor_info
    def predict(self, dataset: Dataset) -> Dataset:
        """ Return Dataset with column(s) added for prediction(s). """
        ...
        dataset.dataf.loc[:, f"prediction_{self.model_name}"] = 0
        feature_df = dataset.get_feature_data
        ...
        # Parse all contents of Dataset to the next pipeline step
        return Dataset(**dataset.__dict__)

<IPython.core.display.Javascript object>